In [23]:
import glob
import os
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D, Flatten
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from sklearn import metrics

In [3]:
def label_creator():
    files = glob.glob('emotion_images/train/*/*')
    len(files)
    files[0:10]
    
    emotions = []
    filename = []
    for i in files:
        emotions.append(i.split('/')[2])
        filename.append(i.split('/')[3])

    outfile = list(zip(files, filename, emotions))

    df = pd.DataFrame(outfile, columns = ['path', 'filename', 'emotion'])
    df['emotion_value'] = df['emotion'].replace(['happy', 'sad', 'fear', 'surprise', 'neutral', 'angry', 'disgust'], [0,1,2,3,4,5,6])
    dummies = pd.get_dummies(df, columns = ['emotion'])
    return dummies
    #return df




In [4]:
df = label_creator()
#dummies

In [5]:
df

,path,filename,emotion_value,emotion_angry,emotion_disgust,emotion_fear,emotion_happy,emotion_neutral,emotion_sad,emotion_surprise
0,emotion_images/train/happy/Training_50449107.jpg,Training_50449107.jpg,0,0,0,0,1,0,0,0
1,emotion_images/train/happy/Training_70433018.jpg,Training_70433018.jpg,0,0,0,0,1,0,0,0
2,emotion_images/train/happy/Training_85610005.jpg,Training_85610005.jpg,0,0,0,0,1,0,0,0
3,emotion_images/train/happy/Training_4460748.jpg,Training_4460748.jpg,0,0,0,0,1,0,0,0
4,emotion_images/train/happy/Training_6312930.jpg,Training_6312930.jpg,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
28704,emotion_images/train/disgust/Training_94788538...,Training_94788538.jpg,6,0,1,0,0,0,0,0
28705,emotion_images/train/disgust/Training_81049148...,Training_81049148.jpg,6,0,1,0,0,0,0,0
28706,emotion_images/train/disgust/Training_28365203...,Training_28365203.jpg,6,0,1,0,0,0,0,0
28707,emotion_images/train/disgust/Training_39197750...,Training_39197750.jpg,6,0,1,0,0,0,0,0


In [6]:
df = df.sample(frac = 1, axis = 0, ignore_index = True)
df

,path,filename,emotion_value,emotion_angry,emotion_disgust,emotion_fear,emotion_happy,emotion_neutral,emotion_sad,emotion_surprise
0,emotion_images/train/angry/Training_75613766.jpg,Training_75613766.jpg,5,1,0,0,0,0,0,0
1,emotion_images/train/angry/Training_70023034.jpg,Training_70023034.jpg,5,1,0,0,0,0,0,0
2,emotion_images/train/fear/Training_6195473.jpg,Training_6195473.jpg,2,0,0,1,0,0,0,0
3,emotion_images/train/happy/Training_82240410.jpg,Training_82240410.jpg,0,0,0,0,1,0,0,0
4,emotion_images/train/sad/Training_4721000.jpg,Training_4721000.jpg,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
28704,emotion_images/train/neutral/Training_30887606...,Training_30887606.jpg,4,0,0,0,0,1,0,0
28705,emotion_images/train/happy/Training_43179020.jpg,Training_43179020.jpg,0,0,0,0,1,0,0,0
28706,emotion_images/train/surprise/Training_2829796...,Training_2829796.jpg,3,0,0,0,0,0,0,1
28707,emotion_images/train/happy/Training_40223311.jpg,Training_40223311.jpg,0,0,0,0,1,0,0,0


In [7]:
imgs = []
for i in df['path']:
    imgs.append(cv2.imread(i, 0))
    
imgs = np.array(imgs)
    
    

In [8]:
imgs.shape

(28709, 48, 48)

In [9]:
imgs = imgs/255

In [10]:
imgs.shape

(28709, 48, 48)

In [11]:
y_train = np.array(df[df.columns[3:]])
y_train

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]], dtype=uint8)

In [12]:
y_train.shape

(28709, 7)

In [13]:
X_train = imgs

In [14]:
X_train.shape

(28709, 48, 48)

In [15]:
X_train = np.expand_dims(X_train, axis = 3)
X_train.shape

(28709, 48, 48, 1)

In [16]:
X_train = X_train/255

In [19]:
single_image = X_train[0]
img_width, img_height, img_depth = single_image.shape
num_classes = y_train.shape[1]

In [24]:
model = Sequential(name = 'DCNN')
model.add(Conv2D(filters = 64, 
                kernel_size = (5,5), 
                input_shape=(img_width, img_height, img_depth),
                padding = 'same',
                activation = 'elu',
                kernel_initializer='he_normal',
                name = 'Conv2D_1'))

model.add(BatchNormalization(name = 'batchnorm_1'))
model.add(Conv2D(
            filters=64,
            kernel_size=(5,5),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_2'))

model.add(BatchNormalization(name='batchnorm_2'))

model.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_1'))



model.add(Flatten(name = 'flatten'))
model.add(Dense(128,
               activation = 'elu',
               kernel_initializer = 'he_normal',
               name = 'dense_1'))
model.add(BatchNormalization(name = 'batchnorm_7'))
model.add(Dropout(.6, name = 'dropout_4'))
model.add(Dense(num_classes,
               activation = 'softmax',
               name = 'output_layer'))

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name='Nadam'),
    metrics = ['accuracy'])